In [ ]:
import re
import requests
from bs4 import BeautifulSoup, Tag
import csv
import sys
from tqdm import tqdm
import warnings
from requests.packages.u
from datetime import datetime
from typing import List, Optional
from pydantic import BaseModel, Field, HttpUrl

rllib3.exceptions import InsecureRequestWarning

# Ignore only the InsecureRequestWarning
warnings.simplefilter("ignore", InsecureRequestWarning)

class SearchResult(BaseModel):
    """
    Individual search result from ITMO news/pages
    """
    title: str
    url: HttpUrl
    description: Optional[str] = None
    date: Optional[datetime] = None
    content: Optional[str] = None

class SearchResponse(BaseModel):
    """
    Complete search response
    """
    query: str
    total_results: int
    results: List[SearchResult]
    search_time: datetime = Field(default_factory=datetime.now)

def clean_article_content(content_block: Tag) -> str:
    """
    Remove unnecessary elements and prepare text:
      - Remove <script> and <style>
      - Convert non-breaking spaces (\xa0) to regular spaces
      - Collapse multiple spaces/newlines
    """
    # 1. Remove <script> and <style> tags
    for unwanted_tag in content_block(["script", "style"]):
        unwanted_tag.decompose()

    # 2. Extract text
    text = content_block.get_text(separator="\n", strip=True)

    # 3. Remove non-breaking spaces
    text = text.replace("\xa0", " ")

    # 4. Collapse multiple newlines and extra spaces
    text = re.sub(r"\n\s*\n+", "\n\n", text)
    text = re.sub(r" {2,}", " ", text)

    # 5. Final strip
    text = text.strip()
    return text

def parse_itmo_article_page(html_content: str) -> dict:
    """
    Extracts data from a single ITMO article page:
      - title
      - publication_datetime (e.g. 2025-01-22T11:48:35+03:00)
      - views
      - authors (list of strings)
      - tags (list of strings)
      - cleaned article_text
    """
    soup = BeautifulSoup(html_content, "html.parser")
    
    # Title
    title_tag = soup.select_one("div.article h1")
    title = title_tag.get_text(strip=True) if title_tag else None

    # Date/time in time[datetime], e.g. 2025-01-22T11:48:35+03:00
    time_tag = soup.select_one("div.news-info-wrapper time")
    publication_datetime_str = time_tag.get("datetime", "").strip() if time_tag else None
    publication_datetime = None
    if publication_datetime_str:
        # Try to parse as an ISO datetime
        # If you need more robust parsing, install/use dateutil.parser.parse(...)
        try:
            publication_datetime = datetime.fromisoformat(publication_datetime_str)
        except ValueError:
            # fallback if the format is not strictly ISO or can't be parsed
            publication_datetime = None

    # Views (example: <span class="icon eye">1049</span>)
    views_span = time_tag.select_one("span.icon.eye") if time_tag else None
    views = views_span.get_text(strip=True) if views_span else None

    # Authors
    authors = []
    for author_block in soup.select(".author-block .author-item"):
        name_tag = author_block.select_one(".about h6 a[rel='author']")
        if name_tag:
            authors.append(name_tag.get_text(strip=True))

    # Tags
    tags = []
    for tag_item in soup.select("ul.tags li a"):
        tags.append(tag_item.get_text(strip=True))

    # Main article text
    article_block = soup.select_one(".content.js-mediator-article")
    article_text = clean_article_content(article_block) if article_block else ""

    return {
        "title": title,
        "publication_datetime": publication_datetime,
        "views": views,
        "authors": authors,
        "tags": tags,
        "article_text": article_text
    }


def parse_itmo_search_page(html_content: str):
    """
    Parses the main ITMO search result page, returning:
      - total_results (string or integer)
      - articles: list of {title, link, snippet, date (string)}
    """
    soup = BeautifulSoup(html_content, "html.parser")

    # Find total results (e.g. "152 результата")
    total_results_str = None
    h2_element = soup.select_one(".weeklyevents h2 span")
    if h2_element:
        total_results_str = h2_element.get_text(strip=True)

    # Convert e.g. "152 результата" -> 152 (if possible)
    total_results = 0
    if total_results_str:
        # Use regex to extract digits
        match = re.search(r"(\d+)", total_results_str)
        if match:
            total_results = int(match.group(1))

    # Parse each result (li.weeklyevent)
    results = []
    for item in soup.select(".weeklyevents ul li.weeklyevent"):
        h4 = item.find("h4")
        if not h4 or not h4.find("a"):
            continue

        # Title & Link
        title_tag = h4.find("a")
        title = title_tag.get_text(strip=True)
        link = title_tag["href"]
        if link.startswith("/"):
            link = "https://news.itmo.ru" + link

        # paragraphs typically contain snippet & date
        paragraphs = item.find_all("p")
        snippet = paragraphs[0].get_text(strip=True) if len(paragraphs) > 0 else None
        date_str = paragraphs[-1].get_text(strip=True) if len(paragraphs) > 1 else None

        results.append({
            "title": title,
            "link": link,
            "snippet": snippet,
            "date": date_str  # date as string for now
        })

    return {
        "total_results": total_results,
        "articles": results
    }

def get_search_results(
    query: str,
    max_articles: int = 5
) -> SearchResponse:
    """
    High-level function that:
      - Searches ITMO news for `query`
      - Parses search results
      - Fetches each article's content
      - Returns a Pydantic SearchResponse
    """
    # Build the search URL
    base_url = "https://news.itmo.ru/ru/search/"
    params = {"search": query}

    # 1. Fetch the search page
    response = requests.get(base_url, params=params, verify=False)
    response.raise_for_status()

    # 2. Parse the search page
    search_data = parse_itmo_search_page(response.text)
    total_found = search_data["total_results"]
    raw_articles = search_data["articles"]

    # 3. For each article (up to max_articles), parse additional data
    results_list = []
    for article_info in raw_articles[:max_articles]:
        url = article_info["link"]
        snippet = article_info["snippet"]
        date_str = article_info["date"]

        # 3a. Fetch & parse the article page
        article_resp = requests.get(url, verify=False)
        article_resp.raise_for_status()
        article_data = parse_itmo_article_page(article_resp.text)

        # 3b. Build a Pydantic SearchResult
        # We'll use the parse_itmo_article_page date if it's valid,
        # otherwise fallback to the date from the search snippet.
        final_date = article_data["publication_datetime"]

        # If parse_itmo_article_page could not parse any date/time, optionally try
        # to parse `date_str` from the snippet. This might be "22.01.2025" or similar.
        # For demonstration, we'll do a simple attempt:
        if not final_date and date_str:
            # Attempt a naive parse of "DD.MM.YYYY" 
            match = re.search(r"(\d{2}\.\d{2}\.\d{4})", date_str)
            if match:
                try:
                    final_date = datetime.strptime(match.group(1), "%d.%m.%Y")
                except ValueError:
                    final_date = None

        # Construct the SearchResult
        search_result = SearchResult(
            title=article_data["title"] or article_info["title"],
            url=url,
            description=snippet,
            date=final_date,
            content=article_data["article_text"]
        )
        results_list.append(search_result)

    # 4. Build & return our SearchResponse
    return SearchResponse(
        query=query,
        total_results=total_found,
        results=results_list
    )

if __name__ == "__main__":
    # Example usage
    from pprint import pprint

    query = "yandex"
    max_articles = 2  # parse first 3 articles in detail
    search_response = get_search_results(query, max_articles)

    # Print out the resulting data
    # Because it's a Pydantic model, we can do .dict() or .json() too
    print("--- SEARCH RESPONSE (dict) ---")
    pprint(search_response.dict())
    
    # Or just show each result nicely
    print("\n--- HUMAN-READABLE OUTPUT ---")
    print(f"Query: {search_response.query}")
    print(f"Total found: {search_response.total_results}")
    for idx, result in enumerate(search_response.results, 1):
        print(f"\nResult {idx}:")
        print(f"  Title: {result.title}")
        print(f"  URL:   {result.url}")
        print(f"  Date:  {result.date}")
        print(f"  Snippet/desc: {result.description}")
        # Show first 200 chars of content
        if result.content:
            print(f"  Content (truncated): {result.content[:200]}...")
        else:
            print("  Content: [empty]")

In [51]:


# Example shortened queries to parse (feel free to adjust or replace them):
queries = [
    "История", "Основание", "Рейтинг", "Титулы", "Миссия",
    "Наука", "Исследования", "Гранты", "Лаборатории", "Публикации",
    "Поступление", "Проходной балл", "Приёмная", "ЕГЭ", "Бюджет",
    "Контракт", "Льготы", "Целевая", "Олимпиады", "Права", "Факультет"
    "Стипендии", "Повышенная", "Соцстипендия", "Студгранты", "Поддержка",
    "Спонсорство", "Президентская", "Международные", "Конкурсы", "Заявка",
    "Магистратура", "Аспирантура", "Докторантура", "Бакалавриат", "Направления",
    "Специальности", "Учплан", "Модули", "Заочка", "Дистанционка", "Новости"
    "Онлайн-курсы", "Допобразование", "Квалификация", "Профпереподг", "Семинары",
    "Летние школы", "Зимние школы", "Вечерние", "Курсы IT", "Компетенции",
    "Партнёры", "Обмен", "Двойной диплом", "Зарубежная", "Академмобильность",
    "Erasmus+", "Иностранцы", "Англопрограммы", "World rankings", "Спорт",
    "Студсовет", "Клубы", "Научные кружки", "Волонтёры", "Студжизнь",
    "Хакатоны", "Проекты", "Конференции", "Стажировки", "Аудитории",
    "Корпуса", "Общежитие", "Проживание", "Матбаза", "Коворкинги",
    "Библиотека", "Электронная", "Инфоресурсы", "Кампус-тур", "Адрес",
    "Транспорт", "Правила", "Академотпуск", "Перевод", "Регламент",
    "Экзамены", "Апелляция", "Э-расписание", "Выпускники", "Вакансии",
    "Трудоустройство", "Компании", "Карьерный центр", "Стартап", "Акселератор",
    "Предприниматель", "Инноватика", "Выдающиеся", "Известные", "Международные"
    "Искуственный интеллект", "Talent Hub", "AI", "ITMO", "ИТМО", "Ректор",
    "Проректор"
]

def main():
    csv_filename = "itmo_search_results.csv"

    # Open CSV file for writing
    with open(csv_filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f, delimiter=";")
        # Header row
        writer.writerow(["Query", "Title", "url", "Date", "Snippet/desc", "content"])

        for query in tqdm(queries):
            try:
                # Call your previously defined function:
                search_response = get_search_results(query, max_articles=40)

                # Print a short summary to console
                print(f"\n=== Query: {query} (Found: {search_response.total_results}) ===")

                for idx, result in enumerate(search_response.results, 1):
                    # Save row to CSV
                    row = [
                        query,
                        result.title,
                        str(result.url),
                        result.date.isoformat() if result.date else "",
                        result.description or "",
                        result.content or ""
                    ]
                    writer.writerow(row)

            except Exception as e:
                # If there's a network error or parse error, print it but continue
                print(f"Error processing query '{query}': {e}", file=sys.stderr)

    print(f"\nAll results have been written to '{csv_filename}'.")

if __name__ == "__main__":
    main()

  1%|          | 1/106 [00:02<04:58,  2.85s/it]


=== Query: История (Found: 440) ===


  2%|▏         | 2/106 [00:05<05:07,  2.96s/it]


=== Query: Основание (Found: 64) ===


  3%|▎         | 3/106 [00:08<04:51,  2.83s/it]

...


=== Query: ИТМО (Found: 6969) ===


 99%|█████████▉| 105/106 [04:04<00:02,  2.11s/it]


=== Query: Ректор (Found: 2685) ===


100%|██████████| 106/106 [04:07<00:00,  2.33s/it]


=== Query: Проректор (Found: 431) ===

All results have been written to 'itmo_search_results.csv'.


In [ ]:
parsed_data = pd.read_csv('itmo_search_results.csv', on_bad_lines='skip', delimiter=';')

In [63]:
parsed_data = parsed_data.rename(columns={'URL': 'url', 'Content': 'content'})
parsed_data['source'] =  parsed_data['url']

In [68]:
parsed_data.to_csv('../data/processed/texts.csv', index=False)

In [70]:
pd.read_csv('../data/processed/texts.csv')

,Query,Title,url,Date,Snippet/desc,content,source
0,История,"«Можно быть прикольным ученым, а не скучным за...",https://news.itmo.ru/ru/science/photonics/news...,2025-01-15T13:17:31+03:00,…й в пиджаке.\r\n\r\n― И как вы решили идти к ...,В школе Алексей Кохановский прочитал автобиогр...,https://news.itmo.ru/ru/science/photonics/news...
1,История,«Инклюзия — логичная эволюция дизайна»: какие ...,https://news.itmo.ru/ru/education/trend/news/1...,2024-11-22T16:01:53+03:00,"…ь на будущем месте работы», — подчеркнул выпу...",Как сегодня создаются отечественные цифровые с...,https://news.itmo.ru/ru/education/trend/news/1...
2,История,Не только сфинксы и Эрмитаж: в Петербурге выбр...,https://news.itmo.ru/ru/education/cooperation/...,2024-10-31T10:00:10+03:00,"…о сфинксами, Исаакиевским собором и Бродским ...",В Петербурге подвели итоги проектной школы по ...,https://news.itmo.ru/ru/education/cooperation/...
3,История,Никакой скучной физкультуры: как «Кронверкские...,https://news.itmo.ru/ru/university_live/social...,2024-10-11T10:02:50+03:00,…Содержание\r\n\r\n\r\n\tКраткаяисторияклуба\r...,Пять раз побеждали во всероссийском конкурсе «...,https://news.itmo.ru/ru/university_live/social...
4,История,"Виртуальные электростанции, но реальное электр...",https://news.itmo.ru/ru/science/cyberphysics/n...,2024-10-08T11:01:13+03:00,"…ается в науку. Еще лучше, если вам удалось по...",Джексон Джон Джусто приехал в Петербург из Тан...,https://news.itmo.ru/ru/science/cyberphysics/n...
...,...,...,...,...,...,...,...
782,Проректор,Нейросеть для учителя: как эффективно использо...,https://news.itmo.ru/ru/education/trend/news/1...,2024-09-04T17:20:13+03:00,…Искусственный интеллект» ИТМО Д…,Искусственный интеллект ― далеко не просто «иг...,https://news.itmo.ru/ru/education/trend/news/1...
783,Проректор,"ITMO CONF 2024: этика, бизнес и образование в ...",https://news.itmo.ru/ru/education/trend/news/1...,2024-09-02T20:25:49+03:00,…ИТМО — это ИИ\r\n\r\nНа открытии конференции ...,В этом году главной темой ITMO CONF стал тезис...,https://news.itmo.ru/ru/education/trend/news/1...
784,Проректор,В ИТМО Хайпарке построят комплекс для фиджитал...,https://news.itmo.ru/ru/startups_and_business/...,2024-06-03T15:22:29+03:00,…бразовательные программы и методические реком...,Представители образования и городской админист...,https://news.itmo.ru/ru/startups_and_business/...
785,Проректор,Поменялись местами: зачем деканы ИТМО неделю р...,https://news.itmo.ru/ru/education/official/new...,2024-05-13T09:30:00+03:00,"…цессами Студофиса, в университете провели экс...",В ИТМО студентам не нужно бегать по инстанциям...,https://news.itmo.ru/ru/education/official/new...
